In [183]:
import tensorflow as tf

In [184]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

testdata=mnist.test.images
testlabels=mnist.test.labels

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [185]:
# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy
# decrease this if you're running out of memory
test_valid_size = 256

# network Parameters
num_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # dropout (probability to keep units)

In [186]:
weights={
    'layer1': tf.Variable(tf.random_normal([5,5,1,32])),
    'layer2': tf.Variable(tf.random_normal([5,5,32,64])),
    'layer3': tf.Variable(tf.random_normal([3,3,64,128])),
    
    'layer4': tf.Variable(tf.random_normal([4*4*128,1024])),
    'layer5': tf.Variable(tf.random_normal([1024,num_classes]))
}

biases={
    'layer1': tf.Variable(tf.random_normal([32])),
    'layer2': tf.Variable(tf.random_normal([64])),
    'layer3': tf.Variable(tf.random_normal([128])),
    'layer4': tf.Variable(tf.random_normal([1024])),
    'layer5': tf.Variable(tf.random_normal([num_classes]))
}

In [187]:
# Define Convolutional Layer Function

def conv2d(x,W,b,strides=1):
    x=tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x=tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [188]:
# Define Max Pooling Layer Function ( k is kernel size )

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [189]:
# Define MODEL

def conv_net(x,weights,biases, dropout):
    # Layer 1
    conv1=conv2d(x,weights['layer1'],biases['layer1'])
    conv1=maxpool2d(conv1,k=2)
    
    # layer 2 
    conv2=conv2d(conv1, weights['layer2'], biases['layer2'])
    conv2=maxpool2d(conv2, k=2)
    
    # layer 3
    conv3=conv2d(conv2,weights['layer3'],biases['layer3'])
    conv3=maxpool2d(conv3,k=2)
    #Fully Connected Layer
    fc1=tf.reshape(conv3,[-1,weights['layer4'].get_shape().as_list()[0]])
    fc1=tf.add(tf.matmul(fc1,weights['layer4']),biases['layer4'])
    fc1=tf.nn.relu(fc1)
    fc1=tf.nn.dropout(fc1,dropout)
    
    #Output layer
    out=tf.add(tf.matmul(fc1,weights['layer5']),biases['layer5'])
    return out

In [190]:
x=tf.placeholder(tf.float32,[None,28,28,1])
y=tf.placeholder(tf.float32,[None,num_classes])
keep_prob=tf.placeholder(tf.float32)

logits=conv_net(x=x,weights=weights,biases=biases,dropout=keep_prob)



cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
accuracy=tf.equal(tf.argmax(logits,axis=1),tf.argmax(y,axis=1))



In [191]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x,batch_y=mnist.train.next_batch(batch_size)
            costtoprint=sess.run(cost,feed_dict={x:batch_x, y:batch_y, keep_prob:dropout})
            sess.run(optimizer,feed_dict={x:batch_x, y:batch_y, keep_prob:dropout})
            accuracytoprint=sum(sess.run(accuracy,feed_dict={x:batch_x, y:batch_y, keep_prob:dropout}))
        print(costtoprint),print(accuracytoprint)
    
    accuracy_full=sum(sess.run(accuracy,feed_dict={x:testdata, y:testlabels, keep_prob:dropout}))
    print('Accuracy on 10,000 samples',accuracy_full)
        

794.568
14
448.596
16
385.965
8
344.798
13
353.435
9
285.655
9
278.774
10
238.553
13
273.908
9
213.654
19
Accuracy on 10,000 samples 956


#### 